In [1]:
from recom.datasets import load_ml_small_rating

# load data
dataset = load_ml_small_rating(need_raw=True)

# load features
ratings = dataset['raw']
ratings_train_dict = dataset['train_dict']
ratings_test_dict = dataset['test_dict']
n_user = dataset['n_user']
n_item = dataset['n_item']
user2ix = dataset['user2ix']
ix2user = dataset['ix2user']
item2ix = dataset['item2ix']
ix2item = dataset['ix2item']

del dataset

print(f'Users: {n_user}, Items: {n_item}. Sparsity: {round(1-len(ratings)/n_user/n_item, 4)}')
print(f'User reduced from {len(user2ix.keys())} to {len(ratings_train_dict.keys())}')

Users: 610, Items: 9724. Sparsity: 0.983
User reduced from 610 to 607


In [60]:
from torch import nn
from torch import Tensor, LongTensor


class ItemAutoEncoder(nn.Module):
    def __init__(self, n_user, hidden_dim=64
                     , encode_actv_func=nn.Sigmoid
                     , decode_actv_func=nn.Identity
                     , dropout=0.05) -> None:
        super(ItemAutoEncoder, self).__init__()
        # encoder: (n_user, hidden_dim)
        self.encoder = nn.Sequential(
            nn.Linear(n_user, hidden_dim, bias=True)
            , encode_actv_func()
        )
        # decoder: (hidden_dim, n_user)
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim, n_user, bias=True)
            , decode_actv_func()
        )
        # dropout to reduce overfitting
        self.dropout = nn.Dropout(p=dropout)

    def pred(self, input):

        hidden = self.dropout(self.encoder(input))
        ratings = self.decoder(hidden)
        
        return ratings

    def forward(self, input):
        """ Only for training.
            We will use mask in this part to control gradients.
        """
        ratings = self.pred(input)
        # we contrain the output value to zero
        # , which, mathematically, can control the gradient of 
        # non-observed rating-related params to zero.
        mask = (input!=0)

        return ratings * mask

In [61]:
def rating_vectorize(rat_dict, n_user, n_item, view='item'):
    from torch import zeros

    rat_mat = zeros(n_user, n_item)
    for u in rat_dict:
        for i in rat_dict[u]:
            rat_mat[u, i] = rat_dict[u][i]
    
    if view == 'item':
        return rat_mat.T
    else:
        return rat_mat

train_mat = rating_vectorize(ratings_train_dict, n_user=n_user, n_item=n_item)
train_mat

tensor([[0.0000, 0.0000, 0.0000,  ..., 2.5000, 3.0000, 5.0000],
        [0.0000, 0.0000, 0.0000,  ..., 2.0000, 0.0000, 0.0000],
        [4.0000, 0.0000, 0.0000,  ..., 2.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [62]:
def autorec_data_loader(rat_dict, n_user, n_item, view='item'
                        , use_sampling=False, obj_size=512
                        , batch_size=256):
    from torch.utils.data import DataLoader
    from random import choices
    from torch import LongTensor

    rating_mat = rating_vectorize(rat_dict, n_user, n_item, view)

    if use_sampling:
        by_axis = n_item if view=='item' else n_user
        rand_ixs = choices(list(range(by_axis)), obj_size)

        return DataLoader(dataset=rating_mat[LongTensor(rand_ixs)]
                          , batch_size=batch_size
                          , shuffle=True)
    else:
        return DataLoader(dataset=rating_mat, batch_size=batch_size, shuffle=True)

dl = autorec_data_loader(ratings_train_dict, n_user, n_item)

In [71]:
from torch.nn import MSELoss

loss = MSELoss()
model = ItemAutoEncoder(610)
data = list(dl)
rmse = (data[0] - model(data[0]))**2

print(data[0])
print(model(data[0]))
print(rmse)

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 3.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 5.0000],
        [5.0000, 0.0000, 0.0000,  ..., 4.0000, 0.0000, 4.5000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])
tensor([[-0.0000,  0.0000, -0.0000,  ..., -0.0000,  0.0000,  0.2000],
        [-0.0000,  0.0000, -0.0000,  ..., -0.0000,  0.0000,  0.1729],
        [-0.5156,  0.0000, -0.0000,  ..., -0.1066,  0.0000,  0.4222],
        ...,
        [-0.0000,  0.0000, -0.0000,  ..., -0.0000,  0.0000,  0.0000],
        [-0.0000,  0.0000, -0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0000,  0.0000, -0.0000,  ..., -0.0000,  0.0000,  0.0000]],
       grad_fn=<MulBackward0>)
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  7.3945],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, 22.6777

In [74]:
rmse.sum(axis=1).shape

torch.Size([256])

In [ ]:
import time
from torch import autograd, LongTensor, device
from torch import optim

model = ItemAutoEncoder(n_user)
opt = optim.SGD(model.parameters(), lr=1, weight_decay=0.01, momentum=0.01)

USE_CUDA = False
N_EPOCHS = 50



if USE_CUDA:
    compute_device = device('cuda')
    model.cuda()
    model.latent_similarity.to('cpu')
else:
    compute_device = device('cpu')

losses = []

t0 = time.time()
for epoch in range(N_EPOCHS):
    train_data = autorec_data_loader(ratings_train_dict, n_user, n_item)

    for i, batch in enumerate(train_data):

        self.zero_grad()

        inputs = autograd.Variable(batch).to(compute_device)

        preds = model(inputs)
        loss = RMSE(rate, self.sigmoid(preds))

        loss.backward()
        opt_fn.step()
        losses.append(loss.data.to(compute_device).tolist())

    if report_interval > 0 \
            and ((epoch+1) % report_interval == 0):
        t1=time.time()
        print(f'Epoch: {epoch+1}, Time: {round(t1-t0,2)}, /Average loss {round(sum(losses[-report_interval:])/report_interval, 5)}')
        t0=time.time()


In [23]:
list(dl)[0].shape

torch.Size([256, 610])

In [24]:
n_user

610

In [13]:
a = Tensor([1,2,3,4,5,6,7,8,9,0])

ixs = LongTensor([1,2])
a[ixs]

tensor([2., 3.])